In [17]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving data.csv to data.csv
User uploaded file "data.csv" with length 18974 bytes


In [3]:
# Install necessary libraries
!pip install pandas numpy matplotlib mplfinance ta-lib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: Could not build wheels for ta-lib, which is required to install pyproject.toml-based projects


In [4]:
pip install ta-lib


  Using cached TA-Lib-0.4.28.tar.gz (357 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for ta-lib
Failed to build ta-lib
ERROR: Could not build wheels for ta-lib, which is required to install pyproject.toml-based projects


In [5]:
# Install necessary libraries
!pip install pandas numpy matplotlib mplfinance pandas_ta


  Using cached mplfinance-0.12.10b0-py3-none-any.whl (75 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=e4386600fa7824fa47126e54a516dbb927839ec1f93ed5e39315bf7ec350e452
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [34]:
import pandas as pd
import numpy as np
data = pd.read_csv('data.csv')
# Ensure correct column names (strip leading/trailing spaces)
data.columns = data.columns.str.strip()
# Clean numerical data: remove commas and convert to float
numeric_columns = ['OPEN', 'HIGH', 'LOW', 'PREV. CLOSE', 'ltp', 'close']
print(data.dtypes)
# Remove commas and convert to float
for column in numeric_columns:
    data[column] = data[column].apply(lambda x: float(x.split()[0].replace(',', '')))
for column in numeric_columns:
    data[column] = pd.to_numeric(data[column], errors='coerce')

# Calculate simple moving averages (SMA)
data['SMA20'] = data['close'].rolling(window=20).mean()  # 20-day SMA
data['SMA50'] = data['close'].rolling(window=50).mean()  # 50-day SMA

# Calculate RSI (Relative Strength Index)
def calculate_rsi(prices, period=14):
    deltas = prices.diff(1).dropna()
    gains = deltas.where(deltas > 0).fillna(0)
    losses = deltas.where(deltas < 0).abs().fillna(0)

    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

data['RSI14'] = calculate_rsi(data['close'], 14)  # 14-day RSI

# Calculate MACD (Moving Average Convergence Divergence)
exp12 = data['close'].ewm(span=12, adjust=False).mean()
exp26 = data['close'].ewm(span=26, adjust=False).mean()
data['MACD'] = exp12 - exp26  # MACD line
data['MACD_signal'] = data['MACD'].ewm(span=9, adjust=False).mean()  # Signal line

# Generate buy and sell signals
# SMA Crossover
data['SMA_buy_signal'] = (data['SMA20'] > data['SMA50']) & (data['SMA20'].shift(1) <= data['SMA50'].shift(1))
data['SMA_sell_signal'] = (data['SMA20'] < data['SMA50']) & (data['SMA20'].shift(1) >= data['SMA50'].shift(1))

# RSI-based signals
data['RSI_buy_signal'] = data['RSI14'] < 30
data['RSI_sell_signal'] = data['RSI14'] > 70

# MACD Crossover
data['MACD_buy_signal'] = (data['MACD'] > data['MACD_signal']) & (data['MACD'].shift(1) <= data['MACD_signal'].shift(1))
data['MACD_sell_signal'] = (data['MACD'] < data['MACD_signal']) & (data['MACD'].shift(1) >= data['MACD_signal'].shift(1))

# Display buy and sell signals
buy_signals = data[data['SMA_buy_signal'] | data['RSI_buy_signal'] | data['MACD_buy_signal']]
sell_signals = data[data['SMA_sell_signal'] | data['RSI_sell_signal'] | data['MACD_sell_signal']]

# Return the buy and sell signals
buy_signals[['close', 'SMA20', 'SMA50', 'RSI14', 'MACD', 'MACD_signal']]  # Buy signals analysis


Date             object
series           object
OPEN             object
HIGH             object
LOW              object
PREV. CLOSE      object
ltp              object
close            object
vwap             object
52W H            object
52W L           float64
VOLUME           object
VALUE            object
No of trades     object
dtype: object


,close,SMA20,SMA50,RSI14,MACD,MACD_signal
10,1266.30,NaN,NaN,NaN,-13.546951,-16.259062
39,1296.05,1276.7375,NaN,60.191638,7.122468,6.753555
49,1328.45,1295.0375,1265.833,65.926558,11.547307,11.095707
54,1351.05,1305.4100,1276.735,62.601432,13.315266,11.526970
62,1407.95,1321.9375,1294.063,70.555655,15.699615,12.221597
76,1247.50,1312.4600,1305.836,21.363555,-20.276818,-7.411961
77,1244.20,1309.0450,1304.853,24.716085,-21.418819,-10.213333
78,1278.10,1306.2675,1304.871,29.093738,-19.365186,-12.043703
81,1285.95,1301.3425,1304.597,41.724138,-13.262203,-13.295034
87,1281.45,1270.6150,1304.112,61.755647,-11.079435,-12.030204


from matplotlib import pyplot as plt
_df_0['close'].plot(kind='hist', bins=20, title='close')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['SMA20'].plot(kind='hist', bins=20, title='SMA20')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['SMA50'].plot(kind='hist', bins=20, title='SMA50')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7.plot(kind='scatter', x='RSI14', y='MACD', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_8['close'].plot(kind='line', figsize=(8, 4), title='close')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_9['SMA20'].plot(kind='line', figsize=(8, 4), title='SMA20')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['SMA50'].plot(kind='line', figsize=(8, 4), title='SMA50')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['RSI14'].plot(kind='line', figsize=(8, 4), title='RSI14')
plt.gca().spines[['top', 'right']].set_visible(False)

In [35]:
buy_signals

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,...,SMA50,RSI14,MACD,MACD_signal,SMA_buy_signal,SMA_sell_signal,RSI_buy_signal,RSI_sell_signal,MACD_buy_signal,MACD_sell_signal
10,10-Apr-2024,EQ,1260.45,1273.45,1252.00,1251.40,1265.05,1266.30,"1,266.95","1,416.30",...,NaN,NaN,-13.546951,-16.259062,False,False,False,False,True,False
39,27-Feb-2024,EQ,1295.00,1305.05,1283.70,1298.25,1293.25,1296.05,"1,295.42","1,416.30",...,NaN,60.191638,7.122468,6.753555,False,False,False,False,True,False
49,13-Feb-2024,EQ,1321.95,1334.80,1300.00,1318.80,1334.80,1328.45,"1,316.49","1,416.30",...,1265.833,65.926558,11.547307,11.095707,False,False,False,False,True,False
54,06-Feb-2024,EQ,1330.00,1362.40,1328.10,1332.75,1355.10,1351.05,"1,349.05","1,416.30",...,1276.735,62.601432,13.315266,11.526970,False,False,False,False,True,False
62,24-Jan-2024,EQ,1372.95,1410.50,1365.55,1366.65,1405.00,1407.95,"1,389.95","1,416.30",...,1294.063,70.555655,15.699615,12.221597,False,False,False,True,True,False
76,04-Jan-2024,EQ,1250.10,1254.70,1231.70,1244.20,1249.00,1247.50,"1,240.56","1,333.05",...,1305.836,21.363555,-20.276818,-7.411961,False,False,True,False,False,False
77,03-Jan-2024,EQ,1265.00,1274.00,1240.00,1278.10,1241.00,1244.20,"1,252.06","1,333.05",...,1304.853,24.716085,-21.418819,-10.213333,False,False,True,False,False,False
78,02-Jan-2024,EQ,1298.00,1299.10,1272.85,1297.15,1279.00,1278.10,"1,279.52","1,333.05",...,1304.871,29.093738,-19.365186,-12.043703,False,False,True,False,False,False
81,28-Dec-2023,EQ,1285.00,1289.00,1270.55,1280.15,1287.00,1285.95,"1,281.95","1,333.05",...,1304.597,41.724138,-13.262203,-13.295034,False,False,False,False,True,False
87,19-Dec-2023,EQ,1300.00,1300.95,1269.20,1291.75,1281.00,1281.45,"1,279.39","1,333.05",...,1304.112,61.755647,-11.079435,-12.030204,False,False,False,False,True,False


In [14]:
data['Date']

KeyError: 'Date'

In [15]:
data

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
0,26-Apr-2024,EQ,"1,250.00","1,347.00","1,241.00","1,190.30","1,280.15","1,278.75","1,303.67","1,416.30",981.05,"2,04,31,926","26,63,64,14,609.85","4,35,299"
1,25-Apr-2024,EQ,"1,185.00","1,193.55","1,172.80","1,185.70","1,190.75","1,190.30","1,185.31","1,416.30",981.05,"33,57,065","3,97,91,63,797.35","1,17,539"
2,24-Apr-2024,EQ,"1,207.95","1,207.95","1,181.50","1,200.25","1,187.40","1,185.70","1,191.99","1,416.30",981.05,"13,75,843","1,63,99,93,965.00","59,828"
3,23-Apr-2024,EQ,"1,218.00","1,218.00","1,198.80","1,208.05","1,200.00","1,200.25","1,203.95","1,416.30",981.05,"14,60,983","1,75,89,50,995.15","56,044"
4,22-Apr-2024,EQ,"1,196.10","1,216.70","1,195.10","1,193.65","1,210.25","1,208.05","1,207.37","1,416.30",981.05,"13,64,143","1,64,70,30,101.50","58,565"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,02-Nov-2023,EQ,"1,125.00","1,130.00","1,113.80","1,125.10","1,117.55","1,115.50","1,120.94","1,320.00",981.05,"24,93,118","2,79,46,35,417.95","82,434"
120,01-Nov-2023,EQ,"1,130.95","1,138.00","1,119.95","1,133.15","1,122.95","1,125.10","1,129.19","1,320.00",981.05,"12,88,497","1,45,49,58,277.25","60,198"
121,31-Oct-2023,EQ,"1,126.90","1,137.15","1,115.55","1,126.90","1,133.00","1,133.15","1,127.11","1,320.00",981.05,"11,94,765","1,34,66,32,374.50","60,751"
122,30-Oct-2023,EQ,"1,120.55","1,138.90","1,115.50","1,120.55","1,126.70","1,126.90","1,127.06","1,320.00",981.05,"22,55,915","2,54,25,44,012.80","71,257"
